In [88]:
import pyodbc

class open_db_connection:
    def __enter__(self):
        cs = 'DRIVER={SQL Server};SERVER=ENSPIRE-PC;DATABASE=EnVisionLite'
        self.connection = pyodbc.connect(cs)
        cursor = self.connection.cursor()
        return cursor
    def __exit__(self, exc_type, exc_value, traceback):
        self.connection.commit()
        self.connection.close()

def execute(sql, *args):        
    with db_cursor() as cursor:
        cursor.execute(sql, *args)
        rows = cursor.fetchall()
    return rows, cursor.description

def get_protocol(protname):        
    with open_db_connection() as cursor:
        cursor.execute('SELECT PlateMap FROM assayProtocol WHERE protName = ?', protname)
        platemap = cursor.fetchone()[0]
    return platemap

def get_map(protname):
    with open_db_connection() as cursor:
        cursor.execute("SELECT CHARINDEX('GROUP Wells=', PlateMap) FROM AssayProtocol WHERE ProtName = ?", protname)
        start = cursor.fetchone()[0] + 12
        cursor.execute("SELECT CHARINDEX('\"', PlateMap, ?) FROM AssayProtocol WHERE ProtName = ?", start+1, protname)
        end = cursor.fetchone()[0]
        sql = 'SELECT SUBSTRING(PlateMap, ?, ?) FROM AssayProtocol WHERE ProtName = ?'
        cursor.execute(sql, start, end-start+1, protname)
        res = cursor.fetchone()[0]
    return res

def set_well(protname, well):
    with open_db_connection() as cursor:
        replaced = '%i-%i [6]' % (well, well)
        cursor.execute("SELECT CHARINDEX('GROUP Wells=', PlateMap) FROM AssayProtocol WHERE ProtName = ?", protname)
        start = cursor.fetchone()[0] + 12
        cursor.execute("SELECT CHARINDEX('\"', PlateMap, ?) FROM AssayProtocol WHERE ProtName = ?", start+1, protname)
        end = cursor.fetchone()[0]

        sql = '''DECLARE @ptrval binary(16)
        SELECT @ptrval = TEXTPTR(PlateMap) FROM AssayProtocol WHERE ProtName = ?
        UPDATETEXT AssayProtocol.PlateMap @ptrval ? ? ?;'''
        cursor.execute(sql, protname, start, end-start-1, replaced)

        sql = 'SELECT SUBSTRING(PlateMap, ?, ?) FROM AssayProtocol WHERE ProtName = ?'
        cursor.execute(sql, start, len(replaced)+2, protname)

        res = cursor.fetchone()[0]
    return res

def well_map(q):
    def char_range(c1, c2):
        for c in range(ord(c1), ord(c2)+1):
            yield chr(c)
    plate = {}
    for n in list('abcdefghijklmnop'):
        plate[n] = [0 for g in range(24)]
    for j in q.replace(' ', '').lower().split(','):
        (b, e) = j.split('-') if ('-' in j) else (j, j)
        for nn in char_range(b[:1], e[:1]):
            for ii in range(int(b[1:]) - 1, int(e[1:])):
                plate[nn][ii] = 1
    out = []
    def add_range(row, start, end):
        rn = (ord(row) - 97) * 24
        out.append('%i-%i [6]' % (rn + start, rn + end))
    for row in list('abcdefghijklmnop'):
        start = None
        for ii, val in enumerate(plate[row]):
#             print(ii, val, start)
            if val and not start:
                start = ii + 1
#                 print('set: ', ii)
            elif not val and start:
                add_range(row, start, ii)
                start = None
        if start:
            add_range(row, start, ii+1)
    return ','.join(out)

def update_protocol(protname, wells, ex, em, step):
    def check(p):
        if type(p) != int:
            raise ValueError('Please use integers.')
        if p < 230 or p > 1000:
            raise ValueError('Monochromator range is between 230nm and 1000nm.')
        else:
            return p
    if ex is None or em is None or step is None:
        raise ValueError('All parameters must be specified.')
    if type(step) != int or step < 1 or step > 750:
        raise ValueError('step must be an integer between 1 and 750')
    
    pp = [None for x in range(8)]
    if type(ex)==int and type(em)==tuple:
        if check(em[0]) - check(ex) < 20:
            raise ValueError('Gap between excitation and emission must be 20nm or more.')
        if check(em[1]) - check(em[0]) < step:
            raise ValueError('Range must not be smaller than step.')
        pp[3] = ex
        pp[4] = em[0]
        pp[5] = em[1]
        pp[6] = step
    elif type(ex)==tuple and type(em)==int:
        if check(em) - check(ex[1]) < 20:
            raise ValueError('Gap between excitation and emission must be 20nm or more.')
        if check(ex[1]) - check(ex[0]) < step:
            raise ValueError('Range must not be smaller than step.')
        pp[7] = em
        pp[0] = ex[0]
        pp[1] = ex[1]
        pp[2] = step
    else:
        raise ValueError('One of ex and em should be set to a range (tuple).')
    
    
    with open_db_connection() as cursor:
        cursor.execute('SELECT PlateMap FROM assayProtocol WHERE protName = ?', protname)
        res = cursor.fetchone()[0]
        if wells is not None:
            new_map = well_map(wells)
            start = res.find('GROUP Wells="') + 13
            end = res.find('"', start)
            res = res[:start] + new_map + res[end:]
        script = res.find('Name="SCANMEAS_MC"')
        start = res.find('Params="', script) + 8
        end = res.find('"', start)
        params = res[start:end].split(',')
        pp_orig = [int(x)//1000 for x in params[1:9]]
        for ii, p in enumerate(pp):
            if p is not None:
                params[ii+1] = str(p * 1000)
            else:
                params[ii+1] = str(- abs(pp_orig[ii] * 1000) )
        res2 = res[:start] + ','.join(params) + res[end:]
        sql = '''DECLARE @ptrval binary(16)
        SELECT @ptrval = TEXTPTR(PlateMap) FROM AssayProtocol WHERE ProtName = ?
        WRITETEXT AssayProtocol.PlateMap @ptrval ?;'''
        cursor.execute(sql, protname, res2)
    

In [90]:
update_protocol('Test', 'A1', ex=440, em=(460, 465), step=5)

In [91]:
update_protocol('Test', 'a1-a4, P24, a9-p10, a12-p13, p1', ex=(400, 500), em=520, step=5)

In [7]:
get_map('Test')

'"1-4 [6],9-10 [6],12-13 [6],25-25 [6],33-34 [6],36-37 [6],57-58 [6],60-61 [6],81-82 [6],84-85 [6],105-106 [6],108-109 [6],129-130 [6],132-133 [6],153-154 [6],156-157 [6],177-178 [6],180-181 [6],201-202 [6],204-205 [6],225-226 [6],228-229 [6],249-250 [6],252-253 [6],273-274 [6],276-277 [6],297-298 [6],300-301 [6],321-322 [6],324-325 [6],345-346 [6],348-349 [6],369-370 [6],372-373 [6],384-384 [6]"'

In [78]:
r2 =well_map('a1-a4, P24, a9-p10, a12-p13, p1')
r2
# well_map('a1-a4, p24')

'1-4 [6],9-10 [6],12-13 [6],33-34 [6],36-37 [6],57-58 [6],60-61 [6],81-82 [6],84-85 [6],105-106 [6],108-109 [6],129-130 [6],132-133 [6],153-154 [6],156-157 [6],177-178 [6],180-181 [6],201-202 [6],204-205 [6],225-226 [6],228-229 [6],249-250 [6],252-253 [6],273-274 [6],276-277 [6],297-298 [6],300-301 [6],321-322 [6],324-325 [6],345-346 [6],348-349 [6],361-361 [6],369-370 [6],372-373 [6],384-384 [6]'

In [73]:
res= get_protocol('Test')
res

'<PLATES Repeat="1" Delay="0" Rows="16" Columns="24"><PLATE Repeat="1" Delay="0" Rows="16" Columns="24" PlateID="10008" Meas_height="-1" Meas_mode="6" Soft_move="0" Rotate="0" Wells="1-384 [6]"><GROUP Wells="1-4 [6],9-10 [6],12-13 [6],25-25 [6],33-34 [6],36-37 [6],57-58 [6],60-61 [6],81-82 [6],84-85 [6],105-106 [6],108-109 [6],129-130 [6],132-133 [6],153-154 [6],156-157 [6],177-178 [6],180-181 [6],201-202 [6],204-205 [6],225-226 [6],228-229 [6],249-250 [6],252-253 [6],273-274 [6],276-277 [6],297-298 [6],300-301 [6],321-322 [6],324-325 [6],345-346 [6],348-349 [6],369-370 [6],372-373 [6],384-384 [6]" Meas_mode="6" WellOrder="1" BLOCK="0" Size="0" UseTime="0" Repeat="1" Delay="0" TotalTime="600"><SCRIPT Name="SCANMEAS_MC" Alias="A" Inc="1" RepeatRange="1 1 1" TimeRange="0 0 1" UseHours="0" Params="4050010,-400000,-500000,-5000,440000,460000,465000,5000,-520000,1,1"/></GROUP></PLATE></PLATES>\r\n'

In [76]:
start = res.find('GROUP Wells="') + 13
end = res.find('"', start)
params_orig = res[start:end]
params_orig

'1-4 [6],9-10 [6],12-13 [6],25-25 [6],33-34 [6],36-37 [6],57-58 [6],60-61 [6],81-82 [6],84-85 [6],105-106 [6],108-109 [6],129-130 [6],132-133 [6],153-154 [6],156-157 [6],177-178 [6],180-181 [6],201-202 [6],204-205 [6],225-226 [6],228-229 [6],249-250 [6],252-253 [6],273-274 [6],276-277 [6],297-298 [6],300-301 [6],321-322 [6],324-325 [6],345-346 [6],348-349 [6],369-370 [6],372-373 [6],384-384 [6]'

In [84]:
params = params_orig.split(',')
params[4] = str(440 * 1000)
','.join(params) == params_orig

True

In [65]:
r1 = get_map('Test')[1:-1]
r1

'1-4 [6],9-10 [6],12-13 [6],25-25 [6],33-34 [6],36-37 [6],57-58 [6],60-61 [6],81-82 [6],84-85 [6],105-106 [6],108-109 [6],129-130 [6],132-133 [6],153-154 [6],156-157 [6],177-178 [6],180-181 [6],201-202 [6],204-205 [6],225-226 [6],228-229 [6],249-250 [6],252-253 [6],273-274 [6],276-277 [6],297-298 [6],300-301 [6],321-322 [6],324-325 [6],345-346 [6],348-349 [6],369-370 [6],372-373 [6],384-384 [6]'

In [66]:
r1 == r2


True